In [4]:
pip install biopandas==0.2.9

     |████████████████████████████████| 264 kB 26.1 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [65]:
!rm qmu_117_ideal_data_latest.pickle

In [1]:
from utility.MoleculeParser import MoleculeData
from utility.QMUQUBO import QMUQUBO
from utility.AnnealerOptimizer import Annealer

import logging

logging.getLogger().setLevel(logging.INFO)

# Step 1: Prepare Data

In [2]:
# initial parameters for experiment data
s3_bucket = f"amazon-braket-1a222675c751" # the name of the bucket
prefix = "annealer-experiment" # the name of the folder in the bucket

raw_path = './molecule-data/117_ideal.mol2' # the mol2 file for this experiment

In [3]:
mol_data = MoleculeData(raw_path, 'qmu')

data_path = mol_data.save("latest")

num_rotation_bond = mol_data.bond_graph.rb_num
print(f"You have loaded the raw molecule data and saved as {data_path}. \n\
This molecule has {num_rotation_bond} rotable bond")

INFO:root:parse mol2 file!
INFO:root:finish save qmu_117_ideal_data_latest.pickle


You have loaded the raw molecule data and saved as ./qmu_117_ideal_data_latest.pickle. 
This molecule has 23 rotable bond


# Step 2: Build Model

In [4]:
# initial the QMUQUBO object
init_param = {}
method = ['pre-calc']

for mt in method:
    if mt == 'pre-calc':
        init_param[mt] = {}
        init_param[mt]['param'] = ['M', 'D', 'A', 'hubo_qubo_val']
    
qmu_qubo = QMUQUBO(mol_data, method, **init_param)

INFO:root:initial pre-calculate for constructing molecule QUBO


In [5]:
# set the parameters for model
model_param = {}
# parameters
num_rotation_bond = mol_data.bond_graph.rb_num

method = 'pre-calc'
model_param[method] = {}
# model_param[method]['M'] = range(1, num_rotation_bond+1)
model_param[method]['M'] = [2,3]
model_param[method]['D'] = [8]
model_param[method]['A'] = [300]
model_param[method]['hubo_qubo_val'] = [200]

qmu_qubo.build_model(**model_param)

INFO:root:['30+31']
INFO:root:['29+30']
INFO:root:['30+31', '29+30']
INFO:root:Construct model for M:2,D:8,A:300,hubo_qubo_val:200 0.0018708268801371256 min
INFO:root:['30+31']
INFO:root:['29+30']
INFO:root:['28+29']
INFO:root:['30+31', '29+30']
INFO:root:['29+30', '28+29']
INFO:root:['30+31', '29+30', '28+29']
INFO:root:Construct model for M:3,D:8,A:300,hubo_qubo_val:200 0.024077177047729492 min


0

In [6]:
# describe the model parameters
model_info = qmu_qubo.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {2, 3}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [7]:
# save the model
model_path = qmu_qubo.save("latest")

print(f"You have built the QUBO model and saved it as {model_path}")

INFO:root:finish save qmu_117_ideal_model_latest.pickle


You have built the QUBO model and saved it as ./qmu_117_ideal_model_latest.pickle


# Step 3: Optimize Configuration

In [8]:
qmu_qubo_optimize = QMUQUBO.load(model_path)

In [9]:
model_info = qmu_qubo_optimize.describe_model()

INFO:root:method: pre-calc
INFO:root:The model_name should be {M}_{D}_{A}_{hubo_qubo_val}
INFO:root:param: M, value {2, 3}
INFO:root:param: D, value {8}
INFO:root:param: A, value {300}
INFO:root:param: hubo_qubo_val, value {200}


In [10]:
# get the model you want to optimize
M = 2
D = 8
A = 300
hubo_qubo_val = 200
model_name = "{}_{}_{}_{}".format(M, D, A, hubo_qubo_val)
method = "pre-calc"

qubo_model = qmu_qubo_optimize.get_model(method, model_name)

In [17]:
method = 'dwave-sa'

optimizer_param = {}
optimizer_param['shots'] = 100

sa_optimizer = Annealer(qubo_model, method, **optimizer_param)

use simulated annealer from dimod


In [18]:
sa_optimize_result = sa_optimizer.fit()

fit() ...
dwave-sa save to local
finish save sa_result.pickle


In [13]:
method = 'dwave-qa'

optimizer_param = {}
optimizer_param['shots'] = 10000
optimizer_param['bucket'] = s3_bucket # the name of the bucket
optimizer_param['prefix'] = prefix # the name of the folder in the bucket
optimizer_param['device'] = "arn:aws:braket:::device/qpu/d-wave/Advantage_system4"
optimizer_param["embed_method"] = "default"

qa_optimizer = Annealer(qubo_model, method, **optimizer_param)

use quantum annealer arn:aws:braket:::device/qpu/d-wave/Advantage_system4 


In [14]:
# not create annealing task, only embedding logic
qa_optimizer.embed()
# create annealing task
qa_optimize_result = qa_optimizer.fit()

fit() ...


INFO:root:_upload_result_json, bucket=amazon-braket-1a222675c751, key=annealer-experiment/2982d4bc-5ad6-4e21-a38d-7943662edeb2/qa_result.pickle


finish save /tmp/qa_result.pickle
dwave-qa save to s3 - 2982d4bc-5ad6-4e21-a38d-7943662edeb2: None


In [15]:
qa_task_id = qa_optimizer.get_task_id()
logging.info(f"task id is {qa_task_id}")

INFO:root:task id is 2982d4bc-5ad6-4e21-a38d-7943662edeb2


In [19]:
logging.info(f"dwave-sa run time {sa_optimize_result['time']}")
logging.info(f"dwave-qa run time {qa_optimize_result['time']}")

INFO:root:dwave-sa run time 17.30202603340149
INFO:root:dwave-qa run time 7.695152997970581


# Step 4: PostProcess Result

#### get time for qpu

In [57]:
from utility.ResultProcess import ResultParser
import time

timestamp = time.strftime("%Y%m%d-%H")

In [18]:
method = "dwave-sa"
sa_param = {}
sa_param["raw_path"] = raw_path
sa_param["data_path"] = data_path

sa_process_result = ResultParser(method, **sa_param)
logging.debug(f"{method} result is {sa_process_result.get_all_result()}")

local_time, _ , _, _= sa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time}")

INFO:root:time for dwave-sa: 
     local time is 0.018619060516357422


_load_raw_result
load simulated annealer raw result
MoleculeData.load()
_init_mol_file
_parse_model_info
_parse_model_info() model_info = {'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_1

In [22]:
sa_atom_pos_data = sa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
sa_process_result.save_mol_file(f"{timestamp}")

INFO:root:!!!!!!!!!!!! found van der waals check fail at 8 and 64 with check: 2.9 v.s. real 2.660067472828462
INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 29 with check: 3.4 v.s. real 2.510724861469293
INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 64 with check: 2.9 v.s. real 2.1757572382965886
INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 65 with check: 2.9 v.s. real 2.1766295252063452
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 29 with check: 3.4 v.s. real 2.3732298519106827
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 64 with check: 2.9 v.s. real 2.456193388151674
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 65 with check: 2.9 v.s. real 1.5181347403969123
INFO:root:!!!!!!!!!!!! found van der waals check fail at 11 and 29 with check: 3.4 v.s. real 3.191020014039399
INFO:root:!!!!!!!!!!!! found van der waals check fail at 11 and 62 with check: 2.9 v.s. real 2.8350548566121256


generate_optimize_pts()
generate_optimize_pts model_info={'model_name': '1_4_300_200', 'rb_name': ['30+31'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x_8_3', '4': 'x_8_4'}, '9': {'1': 'x_9_1', '2': 'x_9_2', '3': 'x_9_3', '4': 'x_9_4'}, '10': {'1': 'x_10_1', '2': 'x_10_2', '3': 'x_10_3', '4': 'x_10_4'}, '11': {'1': 'x_11_1', '2': 'x_11_2', '3': 'x_11_3', '4': 'x_11_4'}, '12': {'1': 'x_12_1', '2': 'x_12_2', '3': 'x_12_3', '4': 'x_12_4'}, '13': {'1': 'x_13_1', '2': 'x_13_2', '3': 'x_13_3', '4': 'x_13_4'}, '14': {'1': 'x_14_1', '2': 'x_14_2', '3': 'x

INFO:root:!!!!!!!!!!!! found van der waals check fail at 49 and 65 with check: 2.4 v.s. real 1.3359641799090274


save_mol_file 20220106-15
finish save ./molecule-data/117_ideal_dwave-sa_20220106-15.mol2 and ./molecule-data/117_ideal_dwave-sa_20220106-15.json


['./molecule-data/117_ideal_dwave-sa_20220106-15.mol2',
 './molecule-data/117_ideal_dwave-sa_20220106-15.json']

In [23]:
sa_process_result.parameters

{'volume': {'initial': 19926.786975170362,
  'optimize': 19758.527817825187,
  'gain': 0.9915561320771465}}

In [58]:
method = "dwave-qa"
qa_param = {}
qa_param["bucket"] = s3_bucket
qa_param["prefix"] = prefix
qa_param["task_id"] = qa_task_id
qa_param["raw_path"] = raw_path
qa_param["data_path"] = data_path

qa_process_result = ResultParser(method, **qa_param)
logging.debug(f"{method} result is {qa_process_result.get_all_result()}")

local_time, task_time, total_time, access_time = qa_process_result.get_time()

logging.info(f"time for {method}: \n \
    local time is {local_time},\n \
    task time is {task_time}, \n \
    qpu total time is {total_time}, \n \
    qpu access time is {access_time}")

_load_raw_result
load quantum annealer raw result
_read_result_obj
_read_result_obj: annealer-experiment/2f6a2b2c-9413-424e-8380-85ed4f366ae6/qa_result.pickle
MoleculeData.load()
_init_mol_file
_parse_model_info
_parse_model_info() model_info = {'model_name': '2_8_300_200', 'rb_name': ['30+31', '29+30'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4', '5': 'x_1_5', '6': 'x_1_6', '7': 'x_1_7', '8': 'x_1_8'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4', '5': 'x_2_5', '6': 'x_2_6', '7': 'x_2_7', '8': 'x_2_8'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4', '5': 'x_3_5', '6': 'x_3_6', '7': 'x_3_7', '8': 'x_3_8'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4', '5': 'x_4_5', '6': 'x_4_6', '7': 'x_4_7', '8': 'x_4_8'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4', '5': 'x_5_5', '6': 'x_5_6', '7': 'x_5_7', '8': 'x_5_8'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4', '5': 'x_6_5', '6': 'x_6_6', '

INFO:root:time for dwave-qa: 
     local time is 7.09752345085144,
     task time is 3.192, 
     qpu total time is 1.057221, 
     qpu access time is 1.0262470000000001


parse quantum annealer result
_read_result_obj
_read_result_obj: annealer-experiment/2f6a2b2c-9413-424e-8380-85ed4f366ae6/results.json


#### parse results

In [59]:
qa_atom_pos_data = qa_process_result.generate_optimize_pts()
# save unfold file for visualization and parameters for experiment: 1. volume value 2. relative improvement
qa_process_result.save_mol_file(f"{timestamp}")

INFO:root:!!!!!!!!!!!! found van der waals check fail at 9 and 29 with check: 3.4 v.s. real 2.903339048406162
INFO:root:!!!!!!!!!!!! found van der waals check fail at 10 and 29 with check: 3.4 v.s. real 3.20725895898663


generate_optimize_pts()
generate_optimize_pts model_info={'model_name': '2_8_300_200', 'rb_name': ['30+31', '29+30'], 'var': {'1': {'1': 'x_1_1', '2': 'x_1_2', '3': 'x_1_3', '4': 'x_1_4', '5': 'x_1_5', '6': 'x_1_6', '7': 'x_1_7', '8': 'x_1_8'}, '2': {'1': 'x_2_1', '2': 'x_2_2', '3': 'x_2_3', '4': 'x_2_4', '5': 'x_2_5', '6': 'x_2_6', '7': 'x_2_7', '8': 'x_2_8'}, '3': {'1': 'x_3_1', '2': 'x_3_2', '3': 'x_3_3', '4': 'x_3_4', '5': 'x_3_5', '6': 'x_3_6', '7': 'x_3_7', '8': 'x_3_8'}, '4': {'1': 'x_4_1', '2': 'x_4_2', '3': 'x_4_3', '4': 'x_4_4', '5': 'x_4_5', '6': 'x_4_6', '7': 'x_4_7', '8': 'x_4_8'}, '5': {'1': 'x_5_1', '2': 'x_5_2', '3': 'x_5_3', '4': 'x_5_4', '5': 'x_5_5', '6': 'x_5_6', '7': 'x_5_7', '8': 'x_5_8'}, '6': {'1': 'x_6_1', '2': 'x_6_2', '3': 'x_6_3', '4': 'x_6_4', '5': 'x_6_5', '6': 'x_6_6', '7': 'x_6_7', '8': 'x_6_8'}, '7': {'1': 'x_7_1', '2': 'x_7_2', '3': 'x_7_3', '4': 'x_7_4', '5': 'x_7_5', '6': 'x_7_6', '7': 'x_7_7', '8': 'x_7_8'}, '8': {'1': 'x_8_1', '2': 'x_8_2', '3': 'x

INFO:root:!!!!!!!!!!!! found van der waals check fail at 64 and 67 with check: 2.4 v.s. real 1.706906634822186
INFO:root:!!!!!!!!!!!! found van der waals check fail at 67 and 76 with check: 2.4 v.s. real 2.2645427485477065


save_mol_file 20220214-07
finish save ./molecule-data/117_ideal_dwave-qa_20220214-07.mol2 and ./molecule-data/117_ideal_dwave-qa_20220214-07.json


['./molecule-data/117_ideal_dwave-qa_20220214-07.mol2',
 './molecule-data/117_ideal_dwave-qa_20220214-07.json']

In [60]:
qa_process_result.parameters

{'volume': {'initial': 19926.786975170362,
  'optimize': 19928.49791776223,
  'gain': 1.0000858614383743}}